In [39]:
import sys
fileDir = "/home/jovyan/notebooks/"
sys.path.append(fileDir)

from utilities import *
import pyspark.sql.functions as F

In [40]:
from pyspark import pandas as ps
import pyspark.sql.types as T

In [41]:
session = create_spark_session("ArangoDB GitHub", SparkConnector.ARANGO)

Added dependencies: 
 ['arangodb-java-driver-shaded-7.1.0.jar', 'arangodb-spark-commons-3.3_2.12-1.5.0.jar', 'arangodb-spark-datasource-3.3_2.12-1.5.0.jar', 'commons-codec-1.11.jar', 'commons-logging-1.2.jar', 'httpclient-4.5.13.jar', 'httpcore-4.4.13.jar', 'jackson-dataformat-velocypack-4.1.0.jar', 'slf4j-api-2.0.7.jar']


In [42]:
# Tutti i commit di uno specifico contributor

options = get_default_options(SparkConnector.ARANGO)
options["query"] = "FOR c in GitContributor \
                    FILTER c._key == 'f08905f3496a7b7d60e0da97307f6ad7594abc92::users.noreply.github.com'\
                    FOR f in outbound c COMMITTED \
                    return {'contributor_name': c.name, 'id_commit': f._key}"

df = spark_read(SparkConnector.ARANGO, session, options=options)
display(df.take(20))

Dataframe loaded from arangodb


[Row(contributor_name='Facebook Community Bot', id_commit='b5651d945cd6f30402b7edbcfb49983159080a47'),
 Row(contributor_name='Facebook Community Bot', id_commit='523ab8333800afbfb169c6fd70ab6611fe07cc2a'),
 Row(contributor_name='Facebook Community Bot', id_commit='9d36375b668875f77ff79732a1f7d916b6a33f9e'),
 Row(contributor_name='Facebook Community Bot', id_commit='f0f83eb3b13bdb6fdfb85b4859709b87d43cf5b6'),
 Row(contributor_name='Facebook Community Bot', id_commit='5b267091ed126ef7fc67ff2dd3749fa3ea5f14ac'),
 Row(contributor_name='Facebook Community Bot', id_commit='1ee7a0a67e77c6b013d2ab983f10408c6dc62047'),
 Row(contributor_name='Facebook Community Bot', id_commit='98a74b0a2ba91a0edbef1802acbd942c4982e084'),
 Row(contributor_name='Facebook Community Bot', id_commit='23fbd312aa80233d4062de28cb950c91786fb3f1'),
 Row(contributor_name='Facebook Community Bot', id_commit='7498e5ce77a7b5c2d3aadf9da0413559f701ed94'),
 Row(contributor_name='Facebook Community Bot', id_commit='ad76eb368ee40f

In [43]:
# Find contributors who have contributed to the largest number of repositories

options = get_default_options(SparkConnector.ARANGO)
options["query"] = "LET distinctValues = (\
                        FOR c IN GitContributor\
                            FOR commit IN OUTBOUND c AUTHOR\
                                FOR r IN OUTBOUND commit BELONGS_TO\
                                    RETURN DISTINCT{c, r})\
                    FOR d in distinctValues\
                        COLLECT contrib = d.c.name WITH COUNT INTO repo_count\
                        SORT repo_count DESC\
                        FILTER repo_count > 1\
                        LIMIT 10\
                        RETURN {contrib, repo_count}"

df = spark_read(SparkConnector.ARANGO, session, options=options)
display(df.take(20))

Dataframe loaded from arangodb


[Row(contrib='Ikko Ashimine', repo_count=4),
 Row(contrib='James George', repo_count=3),
 Row(contrib='dependabot[bot]', repo_count=3),
 Row(contrib='Ronald Eddy Jr', repo_count=3),
 Row(contrib='Michaël De Boey', repo_count=3),
 Row(contrib='Prayag Verma', repo_count=3),
 Row(contrib='Kohei TAKATA', repo_count=3),
 Row(contrib='C. T. Lin', repo_count=3),
 Row(contrib='Haroen Viaene', repo_count=2),
 Row(contrib='Tianyu Yao', repo_count=2)]

In [44]:
session.sparkContext.stop()
session.stop()